In [1]:
import itertools
import logging

import graphcat.notebook

logging.basicConfig(level=logging.DEBUG)

In [2]:
class LoopRepeat(object):
    def __init__(self, sequence):
        self._sequence = sequence
        self._iterator = None

    def __enter__(self):
        self._iterator = iter(self._sequence)
        self._stop = graphcat.common.StopLoop()
        return self._stop
        
    def __exit__(self, exc_type, exc_value, traceback):
        self._iterator = None
        
    def __call__(self, graph, name, inputs):
        if self._iterator is None:
            raise RuntimeError(f"Can't update task {name} outside loop.")
        try:
            next(self._iterator)
            return inputs.getone(None)
        except StopIteration:
            logging.info(f"Task {name} stopping iteration.")
            raise self._stop

            
class LoopIterate(object):
    def __init__(self):
        self._iterator = None
        
    def __enter__(self):
        self._iterator = True
        self._stop = graphcat.common.StopLoop()
        return self._stop
        
    def __exit__(self, exc_type, exc_value, traceback):
        self._iterator = None
        
    def __call__(self, graph, name, inputs):
        if self._iterator is None:
            raise RuntimeError(f"Can't update task {name} outside loop.")
        if self._iterator is True:
            self._iterator = iter(inputs.getone(None))
        try:
            return next(self._iterator)
        except StopIteration:
            logging.info(f"Task {name} stopping iteration.")
            raise self._stop
          
        
class LoopGather(object):
    def __init__(self, loop_begin):
        self._loop_begin = loop_begin
        
    def __call__(self, graph, name, inputs):
        result = []
        with graph.task_fn(self._loop_begin) as stop:
            while True:
                try:
                    graph.mark_unfinished(self._loop_begin)
                    result.append(inputs.getone(None))
                except graphcat.common.StopLoop as e:
                    if e is stop:
                        break
                    raise e
        return result

In [3]:
graph = graphcat.DynamicGraph()
logger = graphcat.Logger(graph)

graph.add_task("A", graphcat.constant("foo"))
graph.add_task("scatter", LoopIterate())
graph.add_task("B", graphcat.passthrough())
graph.add_task("gather", LoopGather("scatter"))

graph.set_links("A", "scatter")
graph.set_links("scatter", "B")
graph.set_links("B", "gather")

graphcat.notebook.display(graph)

In [4]:
try:
    graph.output("scatter")
except Exception:
    pass

INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
ERROR:graphcat.common:Task scatter failed. Exception: Can't update task scatter outside loop.


In [5]:
print(graph.output("gather"))
graphcat.notebook.display(graph)

INFO:graphcat.common:Task gather updating.
INFO:graphcat.common:Task gather executing. Inputs: {None}
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
INFO:graphcat.common:Task A updating.
INFO:graphcat.common:Task A executing. Inputs: {}
INFO:graphcat.common:Task A finished. Output: foo
INFO:graphcat.common:Task scatter finished. Output: f
INFO:graphcat.common:Task B finished. Output: f
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
INFO:graphcat.common:Task scatter finished. Output: o
INFO:graphcat.common:Task B finished. Output: o
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task sc

['f', 'o', 'o']


In [6]:
graph.mark_unfinished("A")
print(graph.output("gather"))
graphcat.notebook.display(graph)

INFO:graphcat.common:Task gather updating.
INFO:graphcat.common:Task gather executing. Inputs: {None}
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
INFO:graphcat.common:Task A updating.
INFO:graphcat.common:Task A executing. Inputs: {}
INFO:graphcat.common:Task A finished. Output: foo
INFO:graphcat.common:Task scatter finished. Output: f
INFO:graphcat.common:Task B finished. Output: f
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
INFO:graphcat.common:Task scatter finished. Output: o
INFO:graphcat.common:Task B finished. Output: o
INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task sc

['f', 'o', 'o']


In [7]:
try:
    graph.output("scatter")
except:
    pass

INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
ERROR:graphcat.common:Task scatter failed. Exception: Can't update task scatter outside loop.


In [8]:
try:
    graph.output("B")
except:
    pass

INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None}
INFO:graphcat.common:Task scatter updating.
INFO:graphcat.common:Task scatter executing. Inputs: {None}
ERROR:graphcat.common:Task scatter failed. Exception: Can't update task scatter outside loop.
ERROR:graphcat.common:Task B failed. Exception: Can't update task scatter outside loop.


In [9]:
print(graph.output("gather"))

INFO:graphcat.common:Task gather updating.


['f', 'o', 'o']


In [10]:
graph = graphcat.DynamicGraph()
logger = graphcat.Logger(graph)

graph.add_task("foo", graphcat.constant("foo"))
graph.add_task("repeat", LoopRepeat(range(2)))
graph.add_task("iterate", LoopIterate())
graph.add_task("innergather", LoopGather("iterate"))
graph.add_task("gather", LoopGather("repeat"))

graph.set_links("foo", "repeat")
graph.set_links("repeat", "iterate")
graph.set_links("iterate", "innergather")
graph.set_links("innergather", "gather")

graphcat.notebook.display(graph)
print(graph.output("gather"))
graphcat.notebook.display(graph)

INFO:graphcat.common:Task gather updating.
INFO:graphcat.common:Task gather executing. Inputs: {None}
INFO:graphcat.common:Task innergather updating.
INFO:graphcat.common:Task innergather executing. Inputs: {None}
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task repeat updating.
INFO:graphcat.common:Task repeat executing. Inputs: {None}
INFO:graphcat.common:Task foo updating.
INFO:graphcat.common:Task foo executing. Inputs: {}
INFO:graphcat.common:Task foo finished. Output: foo
INFO:graphcat.common:Task repeat finished. Output: foo
INFO:graphcat.common:Task iterate finished. Output: f
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task iterate finished. Output: o
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task iterate finished. Output: o
INFO:graphc

[['f', 'o', 'o'], ['f', 'o', 'o']]


In [11]:
graph.set_task("foo", graphcat.constant("bar"))
graphcat.notebook.display(graph)
print(graph.output("gather"))
graphcat.notebook.display(graph)

INFO:graphcat.common:Task gather updating.
INFO:graphcat.common:Task gather executing. Inputs: {None}
INFO:graphcat.common:Task innergather updating.
INFO:graphcat.common:Task innergather executing. Inputs: {None}
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task repeat updating.
INFO:graphcat.common:Task repeat executing. Inputs: {None}
INFO:graphcat.common:Task foo updating.
INFO:graphcat.common:Task foo executing. Inputs: {}
INFO:graphcat.common:Task foo finished. Output: bar
INFO:graphcat.common:Task repeat finished. Output: bar
INFO:graphcat.common:Task iterate finished. Output: b
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task iterate finished. Output: a
INFO:graphcat.common:Task iterate updating.
INFO:graphcat.common:Task iterate executing. Inputs: {None}
INFO:graphcat.common:Task iterate finished. Output: r
INFO:graphc

[['b', 'a', 'r'], ['b', 'a', 'r']]
